In [153]:
import requests
import json
from bs4 import BeautifulSoup
from typing import List
import time
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def extract_urls_lists(search_page_url: str) -> List[str]:
    response = requests.get(search_page_url, params = {})
    soup = BeautifulSoup(response.text, 'html.parser')

    list_of_urls=[]

    for article in soup.find_all('iw-search-card-rendered'):
        for link in article.find_all('a'):
            list_of_urls.append(link.get('href'))
    return list_of_urls

def extract_x_urls(number_of_urls: int=30) -> List[str]:
    immo_base_url = "https://www.immoweb.be/fr/recherche/maison/a-vendre?countries=BE&page="
    immo_end_url = "&orderBy=relevance"
    searchpages_urls_list =[]

    for index in range(1,round(min(334,number_of_urls/30 + 1))):
        full_url = immo_base_url + str(index) + immo_end_url
        searchpages_urls_list.append(full_url)
    
    full_urls_list =[]

    with ThreadPoolExecutor() as pool:
        full_urls_list=list(pool.map(extract_urls_lists, searchpages_urls_list))    
    return sum(full_urls_list,[])


In [144]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
pd.set_option('display.width', None)
def make_one_data_frame(url):
  """
  session = requests.Session()
  retry = Retry(connect=3, backoff_factor=0.5)
  adapter = HTTPAdapter(max_retries=retry)
  session.mount('http://', adapter)
  session.mount('https://', adapter)
  """
  response = requests.get(url)
  test_page = response.text

  dfs = pd.read_html(test_page)

  full_df = pd.concat(dfs).dropna(thresh=2).T
  full_df.columns = full_df.iloc[0]
  full_df = full_df[1:]
  return full_df
  # pd.read_html reads in all tables and returns a list of DataFrames



In [156]:

list_of_urls = extract_x_urls(90)
print('list ready')

with ThreadPoolExecutor() as pool:
        full_df= pd.concat(pool.map(make_one_data_frame, list_of_urls))    

print('done extracting')


#full_df = pd.DataFrame.from_dict(map(dict,dataframes_list))
full_df

list ready


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [161]:
list_of_urls = extract_x_urls(60)
print
with ThreadPoolExecutor() as pool:

    list_of_df= list(pool.map(make_one_data_frame, list_of_urls))

df3= pd.concat(list_of_df, ignore_index=True)

df3

InvalidIndexError: Reindexing only valid with uniquely valued Index objects